<a href="https://colab.research.google.com/github/samueleallen/Scraping-Val-Data/blob/main/ValorantMatchPredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [85]:
import pandas as pd

In [86]:
matches_df = pd.read_csv("overall_game_stats_fixed_fr_fr.csv")
matches_df.head()

,Player,R2.0,ACS,K,D,A,+/- K/D,KAST,ADR,HS%,FK,FD,+/- FK/FD,date,Team,vs Team,result,final_score
0,Derke,1.23,287.0,20,19,4,1,78%,179.0,35%,8.0,4.0,4.0,2/9/2025,Team Vitality,Team Liquid,Win,'3 : 2'
1,Less,1.14,227.0,20,17,5,3,78%,140.0,32%,0.0,1.0,-1.0,2/9/2025,Team Vitality,Team Liquid,Win,'3 : 2'
2,trexx,0.63,161.0,12,19,4,-7,70%,120.0,17%,1.0,4.0,-3.0,2/9/2025,Team Vitality,Team Liquid,Win,'3 : 2'
3,Kicks,0.62,103.0,9,16,7,-7,65%,66.0,25%,0.0,2.0,-2.0,2/9/2025,Team Vitality,Team Liquid,Win,'3 : 2'
4,Sayf,0.37,112.0,9,20,4,-11,52%,70.0,29%,2.0,1.0,1.0,2/9/2025,Team Vitality,Team Liquid,Win,'3 : 2'


In [87]:
matches_df.shape

(9300, 18)

In [88]:
matches_df.dtypes

,0
Player,object
R2.0,float64
ACS,float64
K,int64
D,int64
A,int64
+/- K/D,int64
KAST,object
ADR,float64
HS%,object


In [89]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# Convert date column from object to datetime
matches_df["date"] = pd.to_datetime(matches_df["date"])

# Handle missing values
matches_df.fillna(0, inplace=True)

# Handle percentage columns like 'HS%' and 'KAST'
# Convert to numeric and handle any non-numeric values gracefully
matches_df['HS%'] = pd.to_numeric(matches_df['HS%'], errors='coerce')  # Convert to numeric, handle errors by coercing to NaN
matches_df['KAST'] = pd.to_numeric(matches_df['KAST'], errors='coerce')

# If these columns represent percentages, divide by 100 to scale them
matches_df['HS%'] = matches_df['HS%'] / 100
matches_df['KAST'] = matches_df['KAST'] / 100

# Encode categorical columns like 'Player', 'Team', 'vs Team'
label_encoder = LabelEncoder()
matches_df['Player'] = label_encoder.fit_transform(matches_df['Player'])
matches_df['Team'] = label_encoder.fit_transform(matches_df['Team'])
matches_df['vs Team'] = label_encoder.fit_transform(matches_df['vs Team'])

# Convert 'final_score' to two separate columns for individual scores
matches_df[['final_score_winner', 'final_score_loser']] = matches_df['final_score'].str.split(':', expand=True)

# Convert the split columns to numeric
matches_df['final_score_winner'] = pd.to_numeric(matches_df['final_score_winner'], errors='coerce')
matches_df['final_score_loser'] = pd.to_numeric(matches_df['final_score_loser'], errors='coerce')

# Drop 'final_score' column as it is now split into 'final_score_winner' and 'final_score_loser'
matches_df.drop(columns=['final_score'], inplace=True)

# Convert 'result' to integers (0 for Loss, 1 for Win)
matches_df['result'] = matches_df['result'].replace({'Loss': 0, 'Win': 1})

# Check the result
print(matches_df.dtypes)



Player                         int64
R2.0                         float64
ACS                          float64
K                              int64
D                              int64
A                              int64
+/- K/D                        int64
KAST                         float64
ADR                          float64
HS%                          float64
FK                           float64
FD                           float64
+/- FK/FD                    float64
date                  datetime64[ns]
Team                           int64
vs Team                        int64
result                         int64
final_score_winner           float64
final_score_loser            float64
dtype: object


<ipython-input-89-d8bea29d5d64>:35: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  matches_df['result'] = matches_df['result'].replace({'Loss': 0, 'Win': 1})


In [90]:
from sklearn.ensemble import RandomForestClassifier
# maybe experiment with these values
rf = RandomForestClassifier(n_estimators=50, min_samples_split = 10, random_state = 1)

In [101]:
# Filter the dataset based on date
train = matches_df[matches_df["date"] < '2025-01-01']
test = matches_df[matches_df["date"] >= '2025-01-01']

# Convert 'date' column to numeric (days since the first date)
train['date'] = (train['date'] - train['date'].min()).dt.days
test['date'] = (test['date'] - test['date'].min()).dt.days


<ipython-input-101-11409d05edba>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['date'] = (train['date'] - train['date'].min()).dt.days
<ipython-input-101-11409d05edba>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['date'] = (test['date'] - test['date'].min()).dt.days


In [92]:
# test =  matches_df[matches_df["date"] > '2025-01-01']

In [102]:
predictors = ['K', 'D', 'A', '+/- K/D', 'ADR', 'HS%', 'FK', 'FD', 'KAST', 'Team', 'vs Team', 'date']

In [103]:
print(test[predictors].dtypes)


K            int64
D            int64
A            int64
+/- K/D      int64
ADR        float64
HS%        float64
FK         float64
FD         float64
KAST       float64
Team         int64
vs Team      int64
date         int64
dtype: object


In [104]:
rf.fit(train[predictors], train["result"])

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [105]:
# Now fit the model and predict
rf.fit(train[predictors], train["result"])
preds = rf.predict(test[predictors])

In [106]:
from sklearn.metrics import accuracy_score

In [107]:
acc = accuracy_score(test["result"], preds)

In [108]:
acc

0.48522727272727273

In [109]:
combined = pd.DataFrame(dict(actual=test["result"], predicted=preds))
pd.crosstab(index=combined["actual"], columns=combined["predicted"])

predicted,0,1
actual,,
0,714,606
1,753,567


In [110]:
from sklearn.metrics import precision_score

In [111]:
precision_score(test["result"], preds)

0.4833759590792839